<a href="https://colab.research.google.com/github/Vakhranev/MDB/blob/main/Unmatched%20Lemmas%20Percent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
import pandas as pd

# Load SMARTool dictionaries
df_A1 = pd.read_csv('A1.csv', sep=';')
A1 = df_A1[:754]
df_A2 = pd.read_csv('A2.csv', sep=';')
A2 = df_A2[:1048]
df_B1 = pd.read_csv('B1.csv', sep=';')
B1 = df_B1[:1081]
df_B2 = pd.read_csv('B2.csv', sep=';')
B2 = df_B2[:3331]

slovar_smartool_A1 = A1['Lemma'].str.lower().tolist()
slovar_smartool_A2 = A2['Unnamed: 0'].str.lower().tolist()
slovar_smartool_B1 = B1['Lemma'].str.lower().tolist()
slovar_smartool_B2 = B2['Lemma'].str.lower().tolist()

# Load LM dictionaries
df_nv1 = pd.read_csv('new_vocab_a1.csv', sep=';')
df_nv1 = df_nv1[df_nv1['а'].str.len() >= 2]
nv1 = df_nv1['а'].str.lower().tolist()

df_nv2 = pd.read_csv('new_vocab_a2.csv', sep=';')
df_nv2 = df_nv2[df_nv2['а'].str.len() >= 2]
nv2 = df_nv2['а'].str.lower().tolist()

df_nv3 = pd.read_csv('new_vocab_b1.csv', sep=';')
df_nv3 = df_nv3[df_nv3['а'].str.len() >= 2]
nv3 = df_nv3['а'].str.lower().tolist()

df_nv4 = pd.read_csv('new_vocab_b2.csv', sep=';')
df_nv4 = df_nv4[df_nv4['а'].str.len() >= 2]
nv4 = df_nv4['а'].str.lower().tolist()

slovar_LM_A1 = nv1
slovar_LM_A2 = list(set(nv2) - set(nv1))
slovar_LM_B1 = list(set(nv3) - set(nv2))
slovar_LM_B2 = list(set(nv4) - set(nv3))

# Function to compare and get the number of matches and unique lemmas
def compare_lemmas(slovar1, slovar2):
    set1 = set(slovar1)
    set2 = set(slovar2)
    matches = len(set1.intersection(set2))
    unique_slovar1 = len(set1 - set2)
    unique_slovar2 = len(set2 - set1)
    return matches, unique_slovar1, unique_slovar2

# Compare each level
levels = ['A1', 'A2', 'B1', 'B2']
for level in levels:
    slovar_smartool = globals()[f'slovar_smartool_{level}']
    slovar_LM = globals()[f'slovar_LM_{level}']
    matches, unique_smartool, unique_LM = compare_lemmas(slovar_smartool, slovar_LM)
    print(f"Level {level}:")
    print(f"  Matches: {matches}")
    print(f"  Unique in SMARTool: {unique_smartool}")
    print(f"  Unique in LM: {unique_LM}\n")

Level A1:
  Matches: 721
  Unique in SMARTool: 3
  Unique in LM: 273

Level A2:
  Matches: 497
  Unique in SMARTool: 536
  Unique in LM: 77

Level B1:
  Matches: 927
  Unique in SMARTool: 150
  Unique in LM: 12

Level B2:
  Matches: 3175
  Unique in SMARTool: 2
  Unique in LM: 41



In [3]:
import os
import glob
import pandas as pd
import string
from pymystem3 import Mystem

# Initialize Mystem
mystem = Mystem()

# Load dictionaries for SMARTool
df_A1 = pd.read_csv('A1.csv', sep=';')
df_A1.index = pd.RangeIndex(start=1, stop=len(df_A1) + 1)
slovar1 = df_A1['Lemma'].str.lower().tolist()

df_A2 = pd.read_csv('A2.csv', sep=';')
df_A2.index = pd.RangeIndex(start=1, stop=len(df_A2) + 1)
slovar2 = df_A2['Unnamed: 0'].str.lower().tolist()

df_B1 = pd.read_csv('B1.csv', sep=';')
df_B1.index = pd.RangeIndex(start=1, stop=len(df_B1) + 1)
slovar3 = df_B1['Lemma'].str.lower().tolist()

df_B2 = pd.read_csv('B2.csv', sep=';')
df_B2.index = pd.RangeIndex(start=1, stop=len(df_B2) + 1)
slovar4 = df_B2['Lemma'].str.lower().tolist()

# Load dictionaries for LM (new vocabularies)
df_nv1 = pd.read_csv('new_vocab_a1.csv', sep=';')
df_nv1.index = pd.RangeIndex(start=1, stop=len(df_nv1) + 1)
nv1 = df_nv1[df_nv1['а'].str.len() >= 2]['а'].str.lower().tolist()

df_nv2 = pd.read_csv('new_vocab_a2.csv', sep=';')
df_nv2.index = pd.RangeIndex(start=1, stop=len(df_nv2) + 1)
nv2 = df_nv2[df_nv2['а'].str.len() >= 2]['а'].str.lower().tolist()

df_nv3 = pd.read_csv('new_vocab_b1.csv', sep=';')
df_nv3.index = pd.RangeIndex(start=1, stop=len(df_nv3) + 1)
nv3 = df_nv3[df_nv3['а'].str.len() >= 2]['а'].str.lower().tolist()

df_nv4 = pd.read_csv('new_vocab_b2.csv', sep=';')
df_nv4.index = pd.RangeIndex(start=1, stop=len(df_nv4) + 1)
nv4 = df_nv4[df_nv4['а'].str.len() >= 2]['а'].str.lower().tolist()

# Clean words from quotes in new vocabularies
slovar_nv1 = [word.strip("'") for word in nv1]

# Чистка словаря nv2 от слов, которые уже есть в словаре nv1
slovar_nv2 = [word.strip("'") for word in nv2 if word.strip("'") not in slovar_nv1]

# Чистка словаря nv3 от слов, которые уже есть в словарях nv1 и nv2
slovar_nv3 = [word.strip("'") for word in nv3 if word.strip("'") not in slovar_nv1 and word.strip("'") not in slovar_nv2]

# Чистка словаря nv4 от слов, которые уже есть в словарях nv1, nv2 и nv3
slovar_nv4 = [word.strip("'") for word in nv4 if word.strip("'") not in slovar_nv1 and word.strip("'") not in slovar_nv2 and word.strip("'") not in slovar_nv3]

# Folder path with students' texts
students_path = r"/content/Students_texts"

def lemmatize_text(text):
    words = text.split()
    lemmatized_words = []
    for word in words:
        word = word.strip(string.punctuation)
        if word:
            lemmatized_words.append(word.lower())
    return lemmatized_words

def process_folder(folder_path, slovar1, slovar2, slovar3, slovar4):
    total_unmatched_percentage = 0
    total_unmatched_tokens = 0
    total_files = 0

    for file_path in glob.glob(folder_path + "/*.txt"):
        with open(file_path, "r", encoding='utf-8') as file:
            text = file.read()
            lemmas = lemmatize_text(text)
            unique_lemmas = set(lemmas)

            matched_slovar1 = len(unique_lemmas.intersection(slovar1))
            matched_slovar2 = len(unique_lemmas.intersection(slovar2))
            matched_slovar3 = len(unique_lemmas.intersection(slovar3))
            matched_slovar4 = len(unique_lemmas.intersection(slovar4))

            total_matched = matched_slovar1 + matched_slovar2 + matched_slovar3 + matched_slovar4
            total_unmatched = len(unique_lemmas) - total_matched

            # Ensure that the total number of unmatched lemmas is not negative
            total_unmatched = max(0, total_unmatched)

            # Calculate the percentage of unmatched lemmas
            unmatched_percentage = (total_unmatched / len(unique_lemmas)) * 100 if len(unique_lemmas) > 0 else 0

            total_unmatched_percentage += unmatched_percentage
            total_unmatched_tokens += total_unmatched
            total_files += 1

    avg_unmatched_percentage = total_unmatched_percentage / total_files if total_files > 0 else 0
    avg_unmatched_tokens = total_unmatched_tokens / total_files if total_files > 0 else 0
    return avg_unmatched_percentage, avg_unmatched_tokens

# Process folders A1, A2, B1, B2 with SMARTool dictionaries
print("Processing with SMARTool dictionaries:")
folders = ["A1", "A2", "B1", "B2"]
for folder in folders:
    folder_path = os.path.join(students_path, folder)
    avg_unmatched_percentage, avg_unmatched_tokens = process_folder(folder_path, slovar1, slovar2, slovar3, slovar4)
    print(f"Average unmatched lemmas percentage for folder {folder} (SMARTool): {avg_unmatched_percentage:.2f}%")
    print(f"Average unmatched lemmas count for folder {folder} (SMARTool): {avg_unmatched_tokens:.2f}")

# Process folders A1, A2, B1, B2 with new vocabularies
print("\nProcessing with new vocabularies:")
for folder in folders:
    folder_path = os.path.join(students_path, folder)
    avg_unmatched_percentage, avg_unmatched_tokens = process_folder(folder_path, slovar_nv1, slovar_nv2, slovar_nv3, slovar_nv4)
    print(f"Average unmatched lemmas percentage for folder {folder} (New Vocabularies): {avg_unmatched_percentage:.2f}%")
    print(f"Average unmatched lemmas count for folder {folder} (New Vocabularies): {avg_unmatched_tokens:.2f}")

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


Processing with SMARTool dictionaries:
Average unmatched lemmas percentage for folder A1 (SMARTool): 74.24%
Average unmatched lemmas count for folder A1 (SMARTool): 29.40
Average unmatched lemmas percentage for folder A2 (SMARTool): 75.48%
Average unmatched lemmas count for folder A2 (SMARTool): 65.13
Average unmatched lemmas percentage for folder B1 (SMARTool): 77.31%
Average unmatched lemmas count for folder B1 (SMARTool): 95.61
Average unmatched lemmas percentage for folder B2 (SMARTool): 78.23%
Average unmatched lemmas count for folder B2 (SMARTool): 130.29

Processing with new vocabularies:
Average unmatched lemmas percentage for folder A1 (New Vocabularies): 59.50%
Average unmatched lemmas count for folder A1 (New Vocabularies): 23.13
Average unmatched lemmas percentage for folder A2 (New Vocabularies): 57.41%
Average unmatched lemmas count for folder A2 (New Vocabularies): 50.16
Average unmatched lemmas percentage for folder B1 (New Vocabularies): 60.95%
Average unmatched lemmas

In [ ]:
import os
import glob
import pandas as pd
import string
from pymystem3 import Mystem

# Initialize Mystem
mystem = Mystem()

# Load frequency dictionary for each thousand words
df = pd.read_csv('ru_m3.csv', sep=';')
df.index = pd.RangeIndex(start=1, stop=len(df) + 1)
df = df[['Lemma', 'CEFR']]
mark1 = "a1"
mark2 = "a2"
mark3 = "b1"
mark4 = "b2"
mark5 = "A1"
mark6 = "A2"
mark7 = "B1"
mark8 = "B2"
dc1 = df.query('CEFR == @mark1 | CEFR == @mark5')
dc2 = df.query('CEFR == @mark2 | CEFR == @mark6')
dc3 = df.query('CEFR == @mark3 | CEFR == @mark7')
dc4 = df.query('CEFR == @mark4 | CEFR == @mark8')

l = []
for word in dc1['Lemma']:
    if word not in l:
        l.append(word)
slovar1 = l

l = []
for word in dc2['Lemma']:
    if word not in l:
        l.append(word)
slovar2 = l

l = []
for word in dc3['Lemma']:
    if word not in l:
        l.append(word)
slovar3 = l

l = []
for word in dc4['Lemma']:
    if word not in l:
        l.append(word)
slovar4 = l

df = pd.read_csv('lemmas_levels.csv', sep=',')
df.index = pd.RangeIndex(start=1, stop=len(df) + 1)
mark1 = "1E"
mark2 = "2I"
mark3 = "3AU"
mark4 = "4S"
ll1 = df.query('level == @mark1')
ll2 = df.query('level == @mark2')
ll3 = df.query('level == @mark3')
ll4 = df.query('level == @mark4')

slovar_nv1 = ll1['lemma'].str.strip('\'').tolist()
slovar_nv2 = ll2['lemma'].str.strip('\'').tolist()
slovar_nv3 = ll3['lemma'].str.strip('\'').tolist()
slovar_nv4 = ll4['lemma'].str.strip('\'').tolist()

# Folder path with students' texts
students_path = r"/content/Students_texts"

def lemmatize_text(text):
    words = text.split()
    lemmatized_words = []
    for word in words:
        word = word.strip(string.punctuation)
        if word:
            lemmatized_words.append(word.lower())
    return lemmatized_words

def process_folder(folder_path, slovar1, slovar2, slovar3, slovar4):
    total_unmatched_percentage = 0
    total_unmatched_tokens = 0
    total_files = 0

    for file_path in glob.glob(folder_path + "/*.txt"):
        with open(file_path, "r", encoding='utf-8') as file:
            text = file.read()
            lemmas = lemmatize_text(text)
            unique_lemmas = set(lemmas)

            matched_slovar1 = len(unique_lemmas.intersection(slovar1))
            matched_slovar2 = len(unique_lemmas.intersection(slovar2))
            matched_slovar3 = len(unique_lemmas.intersection(slovar3))
            matched_slovar4 = len(unique_lemmas.intersection(slovar4))

            total_matched = matched_slovar1 + matched_slovar2 + matched_slovar3 + matched_slovar4
            total_unmatched = len(unique_lemmas) - total_matched

            # Ensure that the total number of unmatched lemmas is not negative
            total_unmatched = max(0, total_unmatched)

            # Calculate the percentage of unmatched lemmas
            unmatched_percentage = (total_unmatched / len(unique_lemmas)) * 100 if len(unique_lemmas) > 0 else 0

            total_unmatched_percentage += unmatched_percentage
            total_unmatched_tokens += total_unmatched
            total_files += 1

    avg_unmatched_percentage = total_unmatched_percentage / total_files if total_files > 0 else 0
    avg_unmatched_tokens = total_unmatched_tokens / total_files if total_files > 0 else 0
    return avg_unmatched_percentage, avg_unmatched_tokens

# Process folders A1, A2, B1, B2 with SMARTool dictionaries
print("Processing with KELLY dictionaries:")
folders = ["A1", "A2", "B1", "B2"]
for folder in folders:
    folder_path = os.path.join(students_path, folder)
    avg_unmatched_percentage, avg_unmatched_tokens = process_folder(folder_path, slovar1, slovar2, slovar3, slovar4)
    print(f"Average unmatched lemmas percentage for folder {folder} (SMARTool): {avg_unmatched_percentage:.2f}%")
    print(f"Average unmatched lemmas count for folder {folder} (SMARTool): {avg_unmatched_tokens:.2f}")

# Process folders A1, A2, B1, B2 with new vocabularies
print("\nProcessing with new vocabularies:")
for folder in folders:
    folder_path = os.path.join(students_path, folder)
    avg_unmatched_percentage, avg_unmatched_tokens = process_folder(folder_path, slovar_nv1, slovar_nv2, slovar_nv3, slovar_nv4)
    print(f"Average unmatched lemmas percentage for folder {folder} (New Vocabularies): {avg_unmatched_percentage:.2f}%")
    print(f"Average unmatched lemmas count for folder {folder} (New Vocabularies): {avg_unmatched_tokens:.2f}")

Processing with KELLY dictionaries:
Average unmatched lemmas percentage for folder A1 (SMARTool): 48.07%
Average unmatched lemmas count for folder A1 (SMARTool): 19.48
Average unmatched lemmas percentage for folder A2 (SMARTool): 47.82%
Average unmatched lemmas count for folder A2 (SMARTool): 42.92
Average unmatched lemmas percentage for folder B1 (SMARTool): 53.52%
Average unmatched lemmas count for folder B1 (SMARTool): 67.73
Average unmatched lemmas percentage for folder B2 (SMARTool): 58.75%
Average unmatched lemmas count for folder B2 (SMARTool): 98.98

Processing with new vocabularies:
Average unmatched lemmas percentage for folder A1 (New Vocabularies): 48.82%
Average unmatched lemmas count for folder A1 (New Vocabularies): 19.70
Average unmatched lemmas percentage for folder A2 (New Vocabularies): 49.49%
Average unmatched lemmas count for folder A2 (New Vocabularies): 44.09
Average unmatched lemmas percentage for folder B1 (New Vocabularies): 54.81%
Average unmatched lemmas cou